# Decorators

Yay, decorators! These are kind of hard to WRAP your head around (apologies for the punny foreshadowing). With a little bit of introspection and some object-oriented thinking, they're a breeze.

Below is what a basic template of a decorator may look like. This is the template found on [this page](https://realpython.com/primer-on-python-decorators/), which is an excellent resource. 

In [71]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

There's a lot going on here, but by the end of this notebook you'll be able to confidently use decorators and even make some of your own to help you in the future! We'll go over each concept in detail, but here's a quick line-by-line overview:

*   We start by importing functools, which we'll use to wrap whatever function we're decorating to make it more accessible.
*   We'll define a function called `decorator` (which will be our decorator) that takes an argument `func` (which is a function). In the definition:
  * We'll use a decorator from the functools module we imported and wrap the function we're decorating (for accurate introspection) and use that in the declaration of...
  * Our `wrapper_decorator` inner function! This is where we put code that decorates the function before or after it runs. 
      * We take any arguments in our decorated function and apply the result to a variable called `value`, which is a callable referring to the decorated function. 
  * We then return that callable after executing our decorating code.
  * Lastly we'll return a callable to the wrapper function.

WHAT DOES IT MEAN??? Let's find out!

---
## Functions within functions

Thanks to the way that python is structured, we can do some neat things with functions:

*   We can use functions as arguments to other functions.
*   We can create local functions by defining them in other functions.
*   We can use references to functions as our return values in other functions.

This gives us a lot of flexibility and is a large part of object-oriented programming with python. Don't worry, thinking about it hurt my brain at first, too. Fret not, however, for it's actually quite simple.

First, let's try calling a function *within another function.* 

Define a function called `sum()` that returns the sum of two arguments.

In [72]:
### BEGIN SOLUTION
def sum(a,b):
    return a + b

### END SOLUTION

Now define a function called `square()` with a single parameter and call it something like `math_function`. In the definition, call the parameter function and give it two arguments: '1' and '2'.

Note: We can do this because python's functions are objects, specifically [first-class objects](https://realpython.com/primer-on-python-decorators/), meaning they can be passed as arguments to other functions.

In [73]:
### BEGIN SOLUTION
def square(math_function):
       result = sum(1,2)
       square_root = result **2
       return square_root


    


### END SOLUTION

Finally, call `square()` using `sum()` as your argument. Did you get the right answer?

In [74]:
### BEGIN SOLUTION
square(sum)


### END SOLUTION

9

Now we're going to *define* and then call a function within another function.

Define a function that takes no arguments called outer(). Within that definition, define an [inner function](https://realpython.com/inner-functions-what-are-they-good-for/) called inner() that also takes no arguments, and have it print a string of your choice. Finally, put a call to your inner function at the end of the outer function's definition.

In [75]:
### BEGIN SOLUTION
def outer():
  def inner():
    print("Hello World!")

  inner()

### END SOLUTION

Now try calling the outer function.

In [76]:
### BEGIN SOLUTION
outer()

### END SOLUTION

Hello World!


What would happen if we tried calling the inner function?

In [77]:
# inner()

The inner function isn't available in the global scope. This is an example of [encapsulation](https://www.geeksforgeeks.org/encapsulation-in-python/).

Note: Before defining an inner function, we can always use the `global` keyword to add the name of the function we're defining to the global namespace. That way, if we want to, we can use [`create_global_function([inner_function])`](https://stackoverflow.com/questions/27930038/how-to-define-global-function-in-python) to make it more easily accessible to the top level environment.

Even though functions that are defined within other functions aren't directly accessible from a global scope, functions are still objects with callable methods, and we can access them by assigning their references to variables and calling those variables.

Try calling `sum` without the parentheses.

In [78]:
### BEGIN SOLUTION
sum

### END SOLUTION

<function __main__.sum(a, b)>

What returns is a reference to the function, not the function itself. This will happen whenever we call functions without using parentheses.

Let's make another function called `maybe()`. This function will take a boolean (True/False) as an argument. Within the definition, define two more functions, `yes()` and `no()`. Have the first return "Yes", and the second return "No". 

After those definitions, create a loop that evaluates the argument (the boolean above) and returns a **reference to** the first function if True, the second if False.

In [79]:
### BEGIN SOLUTION
def maybe(boolean):
 def yes():
   return "Yes"
 def no():
   return "No" 

 truth_tables =[boolean]
 for arg in truth_tables:
   
   if arg == True:
      reference = yes
   elif arg == False:
    reference = no
   return reference
  
### END SOLUTION 

Now assign the result of this function (with either True/False as the argument) to a variable and get the callable reference (call the function without parentheses).

In [80]:
### BEGIN SOLUTION
answer = maybe(True)
answer
### END SOLUTION

<function __main__.maybe.<locals>.yes()>

What returns sort of looks like a path, doesn't it? Let's break it down:

The reference uses [dot notation](https://www.askpython.com/python/built-in-methods/dot-notation) and first accesses the [top-level environment](https://docs.python.org/3/library/__main__.html#module-__main__) (`__main__`). Then it finds the globally defined method (`maybe`) and accesses its local variables (`<locals>`). That's where the inner function `[yes/no]` is in this case.

You can call the variable you just assigned and it will act the same as the function to which it refers. Try it now.

In [81]:
### BEGIN SOLUTION
print(answer) 
### END SOLUTION

<function maybe.<locals>.yes at 0x000001FAF610D2D0>


---
## Wrapping it up

A decorator, at its most basic, simply changes how a function runs. It wraps a function in code that will run before and/or after to manipulate its execution. With the concepts we've covered so far, we can make a simple decorator:

In [82]:
def decorator_function(decorated_function):
  def wrapper_function():
    print("I am the wrapper.")
    decorated_function()
    print("I am the wrapper.")
  return wrapper_function

In [83]:
def bland():
  print("I am the decorated function, but this way isn't very sweet.")

With normal syntax, we would called the decorator on the function like this:

In [84]:
bland = decorator_function(bland)
bland()

I am the wrapper.
I am the decorated function, but this way isn't very sweet.
I am the wrapper.


We can use some syntactic sugar here to make things less clunky. In your function declaration, before the definition, you can use pie syntax (@) to apply the decorator, like this:

In [85]:
@decorator_function                         
def sweet():
  print("This way is much sweeter :)")

sweet()

I am the wrapper.
This way is much sweeter :)
I am the wrapper.


What if the function that we want to wrap has [arguments and/or keyword arguments](https://www.geeksforgeeks.org/args-kwargs-python/)? We can use the wildcard `*` in our parameters when defining the wrapper function and when we're calling our wrapped function.

In [86]:
def print_args(func):
  def wrapper(*args, **kwargs):
    print(*args, **kwargs)
    func(*args, **kwargs)
  return wrapper

@print_args
def difference(num1, num2):
  print(abs(num1 - num2))
  
difference(3, 5)

3 5
2


Notice that I used `print()` in the function above instead of `return`. If I want to return values from a wrapped function, I'll have to make sure I return the return value of the function in the wrapper function. Here's our new modification:

In [87]:
def print_args(func):
  def wrapper(*args, **kwargs):
    print(*args, **kwargs)
    func(*args, **kwargs)
    return func(*args, **kwargs)
  return wrapper

@print_args
def difference(num1, num2):
  return abs(num1 - num2)
  
print(difference(47, 10))

47 10
37


Looks pretty good now, right? There's one more thing we need, and it's pretty important. Let's getting the callable reference to `difference()`.

In [88]:
difference

<function __main__.print_args.<locals>.wrapper(*args, **kwargs)>

Here we can see that the function thinks it's actually the inner wrapper function within our decorator. How do we fix that? Let's import the `functools` module and see.

In [89]:
import functools

def print_args(func):
  @functools.wraps(func)
  def wrapper(*args, **kwargs):
    print(*args, **kwargs)
    func(*args, **kwargs)
    return func(*args, **kwargs)
  return wrapper

@print_args
def difference(num1, num2):
  return abs(num1 - num2)
  
print(difference(530, 243))

difference

530 243
287


<function __main__.difference(num1, num2)>

`functools.wraps` will preserve the identity of the wrapped function so that it has accurate [introspection](https://book.pythontips.com/en/latest/object_introspection.html).

We can now see the similarity between our template at the top of the notebook and our function we've made here.

Now, let's make our own quick decorator! To start, import `datetime` from `datetime`.

In [2]:
### BEGIN SOLUTION
from datetime import datetime
### END SOLUTION

Now let's define a "paperboy/papergirl" decorator. This will print "HELLO, LADIES AND GENTLEMEN," followed by the return value of the wrapped function. Make this function, and then right a quick function that returns "It is  `[day_of_the_month]`." Be sure to use the decorator when declaring this second function. Feel free to copy the decorator template at the top of the page.

In [7]:
### BEGIN SOLUTION
def paperboypapergirl(paperboypapergirl):
 def wrapper_function():
   print("HELLO, LADIES AND GENTLEMEN,", end= " ") 
   paperboypapergirl()
 return wrapper_function

def weekday():
  today = datetime.today()
  day_of_week = today.strftime("%A").upper()
  print("IT IS",day_of_week)
weekday = paperboypapergirl(weekday)
weekday()
### END SOLUTION

HELLO,LADIES AND GENTLEMEN, IT IS THURSDAY


Not the **most** useful decorator, but it gives us a good idea of how we can use them. The section below focuses on decorators that'll come in handy in your future coding adventures.

---

## Making and using some useful decorators

What can we actually use decorators for? Let's try making a debugger that will be helpful for you in coding on your own. We'll start with the basic template, but change the name.

In [92]:
def debugger(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        value = func(*args, **kwargs)                           # execute wrapped function
        return value                                            # return wrapped function
    return wrapper_decorator

Now let's add some code to the wrapper function. We want to print the name of the wrapped function and its arguments, and then once we run the function we'll want to print the return value.

In [93]:
def debugger(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # list of args
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # list of kwargs
        signature = ", ".join(args_repr + kwargs_repr)           # join and turn into a string
        print(f"Calling {func.__name__}({signature})")           # print function name and args to console
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # print return value
        return value
    return wrapper_decorator

Try using this code locally. Be sure to import functools, as well. You can then attach it to any function you might want to use for smaller functions that you don't directly call, such as `math` functions or `sys` functions. 

Some more built-in functions that are best used as decorators are:

[`@property`](https://www.programiz.com/python-programming/property),

[`@staticmethod`](https://docs.python.org/3.5/library/functions.html#staticmethod), 

[and](https://www.geeksforgeeks.org/class-method-vs-static-method-python/)

[`@classmethod`](https://docs.python.org/3.5/library/functions.html#classmethod)

Functools and unittest are also very useful modules with decorator functions. The more complex programs you write, the more grateful for these tools you'll be when you're debugging.